<a href="https://colab.research.google.com/github/thihanaung-thnn/notes_Python/blob/main/notes_jose_portilla_lectures/machine_learning_with_python/03_pandas_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas

- [Useful Methods](#useful)
- [Groupby and Multi-level Index](#groupby)
- [Pivot tables](#pivot)
- [Combining DataFrame](#combine)
- [Text Methods](#string)
- [Time Methods](#time)

In [198]:
import numpy as np
import pandas as pd

<a id='useful'></a>
## Useful Methods
- apply()
- describe()
- sort_values()
- corr()
- idxmin() , idxmax()
- value_counts()
- replace()
- unique() , nunique()
- map()
- duplicated and drop_duplicates
- between 
- sample 
- nlargest

In [199]:
df = pd.read_csv('sample_data/tips.csv')
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458


In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   size              244 non-null    int64  
 7   price_per_person  244 non-null    float64
 8   Payer Name        244 non-null    object 
 9   CC Number         244 non-null    int64  
 10  Payment ID        244 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 21.1+ KB


### .apply()
#### Simple function

In [201]:
def last_four(num):
  return str(num)[-4:]

In [202]:
df['CC Number'][0]

3560325168603410

In [203]:
last_four(df['CC Number'][0])

'3410'

In [204]:
df['last_four'] = df['CC Number'].apply(last_four) # column.apply(function_name)
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322


#### More complex functions

In [205]:
df['total_bill'].mean()

19.785942622950824

In [206]:
def expensive(price):
  if price < 10:
    return '$'
  elif price >= 10 and price < 30:
    return '$$'
  else:
    return '$$$'

In [207]:
df['Expensive'] = df['total_bill'].apply(expensive)
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$


#### with lambda

In [208]:
df['total_bill'].apply(lambda x:x*10)

0      169.9
1      103.4
2      210.1
3      236.8
4      245.9
       ...  
239    290.3
240    271.8
241    226.7
242    178.2
243    187.8
Name: total_bill, Length: 244, dtype: float64

#### multiple columns 

In [209]:
def quality(total_bill, tip):
  if tip/total_bill > 0.25:
    return 'Generous'
  else:
    return 'Other'

In [210]:
df['Tip Quality'] = df[['total_bill', 'tip']].apply(
    lambda x: quality(x['total_bill'], x['tip']), axis=1)
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$,Other
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$,Other
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$,Other


In [211]:
# this method is faster and better peformance
df['Tip Quality'] = np.vectorize(quality)(df['total_bill'], df['tip'])
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$,Other
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$,Other
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$,Other
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$,Other
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$,Other


In [212]:
help(np.vectorize)

Help on class vectorize in module numpy:

class vectorize(builtins.object)
 |  vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False, signature=None)
 |  
 |  vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False,
 |            signature=None)
 |  
 |  Generalized function class.
 |  
 |  Define a vectorized function which takes a nested sequence of objects or
 |  numpy arrays as inputs and returns a single numpy array or a tuple of numpy
 |  arrays. The vectorized function evaluates `pyfunc` over successive tuples
 |  of the input arrays like the python map function, except it uses the
 |  broadcasting rules of numpy.
 |  
 |  The data type of the output of `vectorized` is determined by calling
 |  the function with the first element of the input.  This can be avoided
 |  by specifying the `otypes` argument.
 |  
 |  Parameters
 |  ----------
 |  pyfunc : callable
 |      A python function or method.
 |  otypes : str or list of dtypes, optional
 |      The 

### describe

In [213]:
df.describe()

,total_bill,tip,size,price_per_person,CC Number
count,244.000000,244.000000,244.000000,244.000000,2.440000e+02
mean,19.785943,2.998279,2.569672,7.888197,2.563496e+15
std,8.902412,1.383638,0.951100,2.914234,2.369340e+15
min,3.070000,1.000000,1.000000,2.880000,6.040679e+10
25%,13.347500,2.000000,2.000000,5.800000,3.040731e+13
50%,17.795000,2.900000,2.000000,7.255000,3.525318e+15
75%,24.127500,3.562500,3.000000,9.390000,4.553675e+15
max,50.810000,10.000000,6.000000,20.270000,6.596454e+15


In [214]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
total_bill,244.0,1.978594e+01,8.902412e+00,3.070000e+00,1.334750e+01,1.779500e+01,2.412750e+01,5.081000e+01
tip,244.0,2.998279e+00,1.383638e+00,1.000000e+00,2.000000e+00,2.900000e+00,3.562500e+00,1.000000e+01
size,244.0,2.569672e+00,9.510998e-01,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,6.000000e+00
price_per_person,244.0,7.888197e+00,2.914234e+00,2.880000e+00,5.800000e+00,7.255000e+00,9.390000e+00,2.027000e+01
CC Number,244.0,2.563496e+15,2.369340e+15,6.040679e+10,3.040731e+13,3.525318e+15,4.553675e+15,6.596454e+15


### sort_values()

In [215]:
df.sort_values('tip').head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
67,3.07,1.0,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455,5267,$,Generous
236,12.60,1.0,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032,3965,$$,Other
92,5.75,1.0,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780,6392,$,Other


In [216]:
df.sort_values(['tip', 'size']).head(3)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
67,3.07,1.0,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455,5267,$,Generous
111,7.25,1.0,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801,6887,$,Other
92,5.75,1.0,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780,6392,$,Other


### df.corr() for correlation

In [217]:
df.corr()

,total_bill,tip,size,price_per_person,CC Number
total_bill,1.000000,0.675734,0.598315,0.647554,0.104576
tip,0.675734,1.000000,0.489299,0.347405,0.110857
size,0.598315,0.489299,1.000000,-0.175359,-0.030239
price_per_person,0.647554,0.347405,-0.175359,1.000000,0.135240
CC Number,0.104576,0.110857,-0.030239,0.135240,1.000000


In [218]:
df[['total_bill', 'tip']].corr()

,total_bill,tip
total_bill,1.000000,0.675734
tip,0.675734,1.000000


### idxmin, idxmax

In [219]:
df['total_bill'].max()

50.81

In [220]:
df['total_bill'].idxmax() # return index

170

In [221]:
df.iloc[170]

total_bill                     50.81
tip                               10
sex                             Male
smoker                           Yes
day                              Sat
time                          Dinner
size                               3
price_per_person               16.94
Payer Name             Gregory Clark
CC Number           5473850968388236
Payment ID                   Sat1954
last_four                       8236
Expensive                        $$$
Tip Quality                    Other
Name: 170, dtype: object

### Value_counts()

In [222]:
df['sex'].value_counts()

Male      157
Female     87
Name: sex, dtype: int64

In [223]:
df['day'].value_counts().sort_values(ascending=False)

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

### replace()

In [224]:
df['Tip Quality'].value_counts()

Other       234
Generous     10
Name: Tip Quality, dtype: int64

In [225]:
df['Tip Quality'].replace(to_replace='Other', value='OK').value_counts()

OK          234
Generous     10
Name: Tip Quality, dtype: int64

### unique

In [226]:
df['size'].unique()

array([2, 3, 4, 1, 6, 5])

In [227]:
df['size'].nunique()

6

### map

In [228]:
df.time.value_counts()

Dinner    176
Lunch      68
Name: time, dtype: int64

In [229]:
mmap = {'Dinner':'D','Lunch':'L'}
df['time'].map(mmap).value_counts()

D    176
L     68
Name: time, dtype: int64

### duplicates

In [230]:
df.duplicated().value_counts()

False    244
dtype: int64

In [231]:
simple_df = pd.DataFrame([1,2,2],['a','b','c'])
simple_df

,0
a,1
b,2
c,2


In [232]:
simple_df.duplicated()

a    False
b    False
c     True
dtype: bool

In [233]:
simple_df.drop_duplicates()

,0
a,1
b,2


### between

In [234]:
df['total_bill'].between(10,20, inclusive=True).value_counts()

True     130
False    114
Name: total_bill, dtype: int64

In [235]:
df[df['total_bill'].between(10,20,inclusive=True)].head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$,Other
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$,Other
8,15.04,1.96,Male,No,Sun,Dinner,2,7.52,Joseph Mcdonald,3522866365840377,Sun6820,0377,$$,Other
9,14.78,3.23,Male,No,Sun,Dinner,2,7.39,Jerome Abbott,3532124519049786,Sun3775,9786,$$,Other
10,10.27,1.71,Male,No,Sun,Dinner,2,5.14,William Riley,566287581219,Sun2546,1219,$$,Other


### sample

In [236]:
df.sample(4)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
150,14.07,2.50,Male,No,Sun,Dinner,2,7.04,Luke Rice,4813617017359506,Sun8863,9506,$$,Other
208,24.27,2.03,Male,Yes,Sat,Dinner,2,12.14,Jason Carter,4268942915626180,Sat6048,6180,$$,Other
232,11.61,3.39,Male,No,Sat,Dinner,2,5.80,James Taylor,6011482917327995,Sat2124,7995,$$,Generous
183,23.17,6.50,Male,Yes,Sun,Dinner,4,5.79,Dr. Michael James,4718501859162,Sun6059,9162,$$,Generous


In [237]:
df.shape

(244, 14)

In [238]:
df.sample(frac=0.02)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
221,13.42,3.48,Female,Yes,Fri,Lunch,2,6.71,Leslie Kaufman,379437981958785,Fri7511,8785,$$,Generous
26,13.37,2.00,Male,No,Sat,Dinner,2,6.68,Kyle Avery,6531339539615499,Sat6651,5499,$$,Other
186,20.90,3.50,Female,Yes,Sun,Dinner,3,6.97,Heidi Atkinson,4422858423131187,Sun4254,1187,$$,Other
97,12.03,1.50,Male,Yes,Fri,Dinner,2,6.02,Eric Herrera,580116092652,Fri9268,2652,$$,Other
239,29.03,5.92,Male,No,Sat,Dinner,3,9.68,Michael Avila,5296068606052842,Sat2657,2842,$$,Other


### nlargest, nsmallest

In [239]:
df.nlargest(4, 'tip')

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,Expensive,Tip Quality
170,50.81,10.00,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954,8236,$$$,Other
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590,5212,$$$,Other
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239,9808,$$$,Other
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139,0595,$$$,Other


<a id='groupby'></a>
## Groupby and Multi-level index

In [240]:
df = pd.read_csv('mpg.csv')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [241]:
# groupby() creates groupby object waiting for an aggregate method
df.groupby('model_year')

Common options:
* mean(): compute mean
* sum()
* size() : group size
* count() 
* std() 
* var()
* sem() : standard error of mean of groups
* describe()
* first() : first of group values
* last()
* nth() : take nth value
* min()
* max()

https://pandas.pydata.org/docs/reference/groupby.html

In [242]:
avg_year = df.groupby('model_year').mean()
avg_year

,mpg,cylinders,displacement,weight,acceleration,origin
model_year,,,,,,
70,17.689655,6.758621,281.413793,3372.793103,12.948276,1.310345
71,21.250000,5.571429,209.750000,2995.428571,15.142857,1.428571
72,18.714286,5.821429,218.375000,3237.714286,15.125000,1.535714
73,17.100000,6.375000,256.875000,3419.025000,14.312500,1.375000
74,22.703704,5.259259,171.740741,2877.925926,16.203704,1.666667
75,20.266667,5.600000,205.533333,3176.800000,16.050000,1.466667
76,21.573529,5.647059,197.794118,3078.735294,15.941176,1.470588
77,23.375000,5.464286,191.392857,2997.357143,15.435714,1.571429
78,24.061111,5.361111,177.805556,2861.805556,15.805556,1.611111


In [243]:
avg_year.index

Int64Index([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], dtype='int64', name='model_year')

In [244]:
avg_year.columns

Index(['mpg', 'cylinders', 'displacement', 'weight', 'acceleration', 'origin'], dtype='object')

In [245]:
avg_year['mpg']

model_year
70    17.689655
71    21.250000
72    18.714286
73    17.100000
74    22.703704
75    20.266667
76    21.573529
77    23.375000
78    24.061111
79    25.093103
80    33.696552
81    30.334483
82    31.709677
Name: mpg, dtype: float64

In [246]:
df.groupby('model_year').describe()

mpg                                     ... origin                    
           count       mean       std   min     25%  ...    min  25%  50%  75%  max
model_year                                           ...                           
70          29.0  17.689655  5.339231   9.0  14.000  ...    1.0  1.0  1.0  1.0  3.0
71          28.0  21.250000  6.591942  12.0  15.500  ...    1.0  1.0  1.0  2.0  3.0
72          28.0  18.714286  5.435529  11.0  13.750  ...    1.0  1.0  1.0  2.0  3.0
73          40.0  17.100000  4.700245  11.0  13.000  ...    1.0  1.0  1.0  2.0  3.0
74          27.0  22.703704  6.420010  13.0  16.000  ...    1.0  1.0  1.0  2.0  3.0
75          30.0  20.266667  4.940566  13.0  16.000  ...    1.0  1.0  1.0  2.0  3.0
76          34.0  21.573529  5.889297  13.0  16.750  ...    1.0  1.0  1.0  2.0  3.0
77          28.0  23.375000  6.675862  15.0  17.375  ...    1.0  1.0  1.0  2.0  3.0
78          36.0  24.061111  6.898044  16.2  19.350  ...    1.0  1.0  1.0  2.0  3.0
79          29.0  25.093103  6.794217  15.5  19.200  ...    1.0  1.0  1.0  1.0  3.0
80          29.0  33.696552  7.037983  19.1  29.800  ...    1.0  2.0  2.0  3.0  3.0
81          29.0  30.334483  5.591465  17.6  26.600  ...    1.0  1.0  2.0  3.0  3.0
82          31.0  31.709677  5.392548  22.0  27.000  ...    1.0  1.0  1.0  3.0  3.0

[13 rows x 48 columns]

In [247]:
year_cyl = df.groupby(['model_year','cylinders']).mean()
year_cyl.head(6)

mpg  displacement  ...  acceleration    origin
model_year cylinders                           ...                        
70         4          25.285714    107.000000  ...     16.000000  2.285714
           6          20.500000    199.000000  ...     15.500000  1.000000
           8          14.111111    367.555556  ...     11.194444  1.000000
71         4          27.461538    101.846154  ...     16.961538  1.923077
           6          18.000000    243.375000  ...     14.750000  1.000000
           8          13.428571    371.714286  ...     12.214286  1.000000

[6 rows x 5 columns]

In [248]:
year_cyl.index # multi-index

MultiIndex([(70, 4),
            (70, 6),
            (70, 8),
            (71, 4),
            (71, 6),
            (71, 8),
            (72, 3),
            (72, 4),
            (72, 8),
            (73, 3),
            (73, 4),
            (73, 6),
            (73, 8),
            (74, 4),
            (74, 6),
            (74, 8),
            (75, 4),
            (75, 6),
            (75, 8),
            (76, 4),
            (76, 6),
            (76, 8),
            (77, 3),
            (77, 4),
            (77, 6),
            (77, 8),
            (78, 4),
            (78, 5),
            (78, 6),
            (78, 8),
            (79, 4),
            (79, 5),
            (79, 6),
            (79, 8),
            (80, 3),
            (80, 4),
            (80, 5),
            (80, 6),
            (81, 4),
            (81, 6),
            (81, 8),
            (82, 4),
            (82, 6)],
           names=['model_year', 'cylinders'])

In [249]:
year_cyl.index.levels

FrozenList([[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], [3, 4, 5, 6, 8]])

In [250]:
year_cyl.index.names

FrozenList(['model_year', 'cylinders'])

In [251]:
year_cyl.loc[70]

,mpg,displacement,weight,acceleration,origin
cylinders,,,,,
4,25.285714,107.000000,2292.571429,16.000000,2.285714
6,20.500000,199.000000,2710.500000,15.500000,1.000000
8,14.111111,367.555556,3940.055556,11.194444,1.000000


In [252]:
year_cyl.loc[[70,75]] # grab from outside ## from model_year

mpg  displacement  ...  acceleration    origin
model_year cylinders                           ...                        
70         4          25.285714    107.000000  ...     16.000000  2.285714
           6          20.500000    199.000000  ...     15.500000  1.000000
           8          14.111111    367.555556  ...     11.194444  1.000000
75         4          25.250000    114.833333  ...     15.833333  2.166667
           6          17.583333    233.750000  ...     17.708333  1.000000
           8          15.666667    330.500000  ...     13.166667  1.000000

[6 rows x 5 columns]

In [253]:
year_cyl.loc[(70,8)] # grab single row

mpg               14.111111
displacement     367.555556
weight          3940.055556
acceleration      11.194444
origin             1.000000
Name: (70, 8), dtype: float64

In [254]:
# grab based on cross_section with .xs()
year_cyl.xs(key=70, axis=0, level='model_year')

,mpg,displacement,weight,acceleration,origin
cylinders,,,,,
4,25.285714,107.000000,2292.571429,16.000000,2.285714
6,20.500000,199.000000,2710.500000,15.500000,1.000000
8,14.111111,367.555556,3940.055556,11.194444,1.000000


In [255]:
year_cyl.xs(key=4, axis=0, level='cylinders')

,mpg,displacement,weight,acceleration,origin
model_year,,,,,
70,25.285714,107.000000,2292.571429,16.000000,2.285714
71,27.461538,101.846154,2056.384615,16.961538,1.923077
72,23.428571,111.535714,2382.642857,17.214286,1.928571
73,22.727273,109.272727,2338.090909,17.136364,2.000000
74,27.800000,96.533333,2151.466667,16.400000,2.200000
75,25.250000,114.833333,2489.250000,15.833333,2.166667
76,26.766667,106.333333,2306.600000,16.866667,1.866667
77,29.107143,106.500000,2205.071429,16.064286,1.857143
78,29.576471,112.117647,2296.764706,16.282353,2.117647


In [256]:
# easier to filter out values before groupby
df[df['cylinders'].isin([4,6])].groupby(['model_year','cylinders']).mean().head()

mpg  displacement  ...  acceleration    origin
model_year cylinders                           ...                        
70         4          25.285714    107.000000  ...     16.000000  2.285714
           6          20.500000    199.000000  ...     15.500000  1.000000
71         4          27.461538    101.846154  ...     16.961538  1.923077
           6          18.000000    243.375000  ...     14.750000  1.000000
72         4          23.428571    111.535714  ...     17.214286  1.928571

[5 rows x 5 columns]

In [257]:
# swap levels
year_cyl.swaplevel().head()

,,mpg,displacement,weight,acceleration,origin
cylinders,model_year,,,,,
4,70,25.285714,107.000000,2292.571429,16.000000,2.285714
6,70,20.500000,199.000000,2710.500000,15.500000,1.000000
8,70,14.111111,367.555556,3940.055556,11.194444,1.000000
4,71,27.461538,101.846154,2056.384615,16.961538,1.923077
6,71,18.000000,243.375000,3171.875000,14.750000,1.000000


In [258]:
# sort multi-index
year_cyl.sort_index(level='model_year', ascending=False).head(6)

mpg  displacement  ...  acceleration    origin
model_year cylinders                           ...                        
82         6          28.333333    225.000000  ...     16.033333  1.000000
           4          32.071429    118.571429  ...     16.703571  1.714286
81         8          26.600000    350.000000  ...     19.000000  1.000000
           6          23.428571    184.000000  ...     15.442857  1.714286
           4          32.814286    108.857143  ...     16.466667  2.095238
80         6          25.900000    196.500000  ...     15.050000  2.000000

[6 rows x 5 columns]

In [259]:
year_cyl.sort_index(level='cylinders', ascending=False).head(6)

,,mpg,displacement,weight,acceleration,origin
model_year,cylinders,,,,,
81,8,26.600000,350.000000,3725.000000,19.000000,1.0
79,8,18.630000,321.400000,3862.900000,15.400000,1.0
78,8,19.050000,300.833333,3563.333333,13.266667,1.0
77,8,16.000000,335.750000,4177.500000,13.662500,1.0
76,8,14.666667,324.000000,4064.666667,13.222222,1.0
75,8,15.666667,330.500000,4108.833333,13.166667,1.0


### agg( ) method

In [260]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [261]:
df.agg(['mean','median'])

,mpg,cylinders,displacement,weight,acceleration,model_year,origin
mean,23.514573,5.454774,193.425879,2970.424623,15.56809,76.01005,1.572864
median,23.000000,4.000000,148.500000,2803.500000,15.50000,76.00000,1.000000


In [262]:
df.agg(['sum','mean'])[['mpg','weight']]

,mpg,weight
sum,9358.800000,1.182229e+06
mean,23.514573,2.970425e+03


In [263]:
# agg per columns 
df.agg({'mpg':['median', 'mean'], 'weight':['sum','mean']})

,mpg,weight
mean,23.514573,2.970425e+03
median,23.000000,NaN
sum,NaN,1.182229e+06


In [264]:
# agg with groupby
df.groupby('cylinders').agg({'mpg':['mean','median'], 'weight':['sum','std']})

mpg         weight            
                mean median     sum         std
cylinders                                      
3          20.550000  20.25    9594  247.566153
4          29.286765  28.25  470858  347.093936
5          27.366667  25.40    9310  374.343870
6          19.985714  19.00  268651  332.297419
8          14.963107  14.00  423816  448.833159

<a id='pivot'></a>
## Pivot Tables

In [265]:
# help(pd.pivot)

In [266]:
df = pd.read_csv('Sales_Funnel_CRM.csv')
df.head()

,Account Number,Company,Contact,Account Manager,Product,Licenses,Sale Price,Status
0,2123398,Google,Larry Pager,Edward Thorp,Analytics,150,2100000,Presented
1,2123398,Google,Larry Pager,Edward Thorp,Prediction,150,700000,Presented
2,2123398,Google,Larry Pager,Edward Thorp,Tracking,300,350000,Under Review
3,2192650,BOBO,Larry Pager,Edward Thorp,Analytics,150,2450000,Lost
4,420496,IKEA,Elon Tusk,Edward Thorp,Analytics,300,4550000,Won


In [267]:
license = df[['Company', 'Product', 'Licenses']]
license.head(10)

,Company,Product,Licenses
0,Google,Analytics,150
1,Google,Prediction,150
2,Google,Tracking,300
3,BOBO,Analytics,150
4,IKEA,Analytics,300
5,Tesla Inc.,Analytics,300
6,Tesla Inc.,Prediction,150
7,Microsoft,Tracking,300
8,Walmart,Analytics,150
9,Apple,Analytics,300


In [268]:
pd.pivot(data = license, index = 'Company', columns = 'Product', values = 'Licenses')

Product,Analytics,GPS Positioning,Prediction,Tracking
Company,,,,
Google,150.0,NaN,150.0,300.0
ATT,NaN,NaN,150.0,150.0
Apple,300.0,NaN,NaN,NaN
BOBO,150.0,NaN,NaN,NaN
CVS Health,NaN,NaN,NaN,450.0
Cisco,300.0,300.0,NaN,NaN
Exxon Mobile,150.0,NaN,NaN,NaN
IKEA,300.0,NaN,NaN,NaN
Microsoft,NaN,NaN,NaN,300.0


In [269]:
# pivot_table method 
pd.pivot_table(df, index='Company', aggfunc='sum')

,Account Number,Licenses,Sale Price
Company,,,
Google,6370194,600,3150000
ATT,1396064,300,1050000
Apple,405886,300,4550000
BOBO,2192650,150,2450000
CVS Health,902797,450,490000
Cisco,4338998,600,4900000
Exxon Mobile,470248,150,2100000
IKEA,420496,300,4550000
Microsoft,1216870,300,350000


In [270]:
pd.pivot_table(df, index='Company', aggfunc='sum')[['Licenses', 'Sale Price']]

,Licenses,Sale Price
Company,,
Google,600,3150000
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
IKEA,300,4550000
Microsoft,300,350000


In [271]:
pd.pivot_table(df, index="Company", aggfunc='sum', values=['Licenses', 'Sale Price']) # same as above

,Licenses,Sale Price
Company,,
Google,600,3150000
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
IKEA,300,4550000
Microsoft,300,350000


In [272]:
df.groupby('Company').sum()[['Licenses', 'Sale Price']]

,Licenses,Sale Price
Company,,
Google,600,3150000
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
IKEA,300,4550000
Microsoft,300,350000


In [273]:
pd.pivot_table(df, index=['Account Manager', 'Contact'], values = ['Sale Price'], aggfunc = 'sum')

Sale Price
Account Manager Contact                 
Claude Shannon  Cindy Phoner     7700000
                Emma Gordian    12390000
Edward Thorp    Elon Tusk        8050000
                Larry Pager      5600000
                Will Grates      2800000

In [274]:
pd.pivot_table(df, index=['Account Manager', 'Contact'],
               values = ['Sale Price'], columns = ['Product'],
               aggfunc=[np.sum, np.mean], fill_value = 0)

sum                  ...       mean         
                             Sale Price                  ... Sale Price         
Product                       Analytics GPS Positioning  ... Prediction Tracking
Account Manager Contact                                  ...                    
Claude Shannon  Cindy Phoner    6650000               0  ...     700000   350000
                Emma Gordian   11550000          350000  ...          0   490000
Edward Thorp    Elon Tusk       7350000               0  ...     700000        0
                Larry Pager     4550000               0  ...     700000   350000
                Will Grates     2450000               0  ...          0   350000

[5 rows x 8 columns]

In [275]:
pd.pivot_table(df, index=['Account Manager', 'Contact', 'Product'],
               values = ['Sale Price','Licenses'], 
               aggfunc=[np.sum],fill_value = 0, margins = True)

sum           
                                             Licenses Sale Price
Account Manager Contact      Product                            
Claude Shannon  Cindy Phoner Analytics            450    6650000
                             Prediction           150     700000
                             Tracking             150     350000
                Emma Gordian Analytics           1050   11550000
                             GPS Positioning      300     350000
                             Tracking             450     490000
Edward Thorp    Elon Tusk    Analytics            600    7350000
                             Prediction           150     700000
                Larry Pager  Analytics            300    4550000
                             Prediction           150     700000
                             Tracking             300     350000
                Will Grates  Analytics            150    2450000
                             Tracking             300     350000
All                                              4500   36540000

In [276]:
pd.pivot_table(df, index = ['Account Manager', 'Status'],
               values = ['Sale Price'], aggfunc = [np.sum], 
               fill_value = 0, margins = True)

sum
                             Sale Price
Account Manager Status                 
Claude Shannon  Lost            4550000
                Presented       3150000
                Under Review     350000
                Won            12040000
Edward Thorp    Lost            4900000
                Presented       3500000
                Under Review    3500000
                Won             4550000
All                            36540000

<a id='combine'></a>
## Combining DataFrames

### Concatenation

In [277]:
# create 2 df
A, B, C, D = [[i + str(x) for x in range(4)] for i in 'ABCD']
df_1 = pd.DataFrame({'A':A, 'B':B})
df_2 = pd.DataFrame({'C':C, 'D':D})
df_1


,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [278]:
df_2

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


In [279]:
# Axis = 0, concatenate along rows
pd.concat([df_1, df_2], axis = 0)

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


In [280]:
# axis = 1 , concatenate along columns 
pd.concat([df_1, df_2], axis = 1)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### Merge

In [281]:
# help(pd.merge)
reg = pd.DataFrame({'reg_id':[1,2,3,4],'Name':['Paul','Gilbert','Steve','Vai']})
login = pd.DataFrame({'log_id':[1,2,3,4],'Name':['Guthrie','Govan','Paul','Gilbert']})
reg

,reg_id,Name
0,1,Paul
1,2,Gilbert
2,3,Steve
3,4,Vai


In [282]:
login

,log_id,Name
0,1,Guthrie
1,2,Govan
2,3,Paul
3,4,Gilbert


### Inner Join, Left Join , Right Join, Outer Join

In [283]:
pd.merge(reg, login, how='inner', on='Name')

,reg_id,Name,log_id
0,1,Paul,3
1,2,Gilbert,4


In [284]:
pd.merge(reg, login, how='outer',left_on = 'reg_id',right_on = 'log_id') 
# example for different column names, although this example doesn't make sense

,reg_id,Name_x,log_id,Name_y
0,1,Paul,1,Guthrie
1,2,Gilbert,2,Govan
2,3,Steve,3,Paul
3,4,Vai,4,Gilbert


In [285]:
pd.merge(reg, login, how = 'left', on = 'Name')

,reg_id,Name,log_id
0,1,Paul,3.0
1,2,Gilbert,4.0
2,3,Steve,NaN
3,4,Vai,NaN


In [286]:
pd.merge(reg, login, how = 'right', on = 'Name')

,reg_id,Name,log_id
0,NaN,Guthrie,1
1,NaN,Govan,2
2,1.0,Paul,3
3,2.0,Gilbert,4


In [287]:
pd.merge(reg, login, how = 'outer', on = 'Name')

,reg_id,Name,log_id
0,1.0,Paul,3.0
1,2.0,Gilbert,4.0
2,3.0,Steve,NaN
3,4.0,Vai,NaN
4,NaN,Guthrie,1.0
5,NaN,Govan,2.0


### Join on Index or column

In [288]:
reg = reg.set_index('Name')
reg

,reg_id
Name,
Paul,1
Gilbert,2
Steve,3
Vai,4


In [289]:
pd.merge(reg, login, left_index=True, right_on='Name')

,reg_id,log_id,Name
2,1,3,Paul
3,2,4,Gilbert


In [290]:
pd.merge(login, reg, right_index=True, left_on='Name')

,log_id,Name,reg_id
2,3,Paul,1
3,4,Gilbert,2


In [291]:
reg = reg.reset_index()
reg

,Name,reg_id
0,Paul,1
1,Gilbert,2
2,Steve,3
3,Vai,4


In [292]:
reg.columns = ['name','id']
login.columns = ['id','name']
reg

,name,id
0,Paul,1
1,Gilbert,2
2,Steve,3
3,Vai,4


In [293]:
login

,id,name
0,1,Guthrie
1,2,Govan
2,3,Paul
3,4,Gilbert


In [294]:
pd.merge(reg, login, on = 'name')

,name,id_x,id_y
0,Paul,1,3
1,Gilbert,2,4


In [295]:
# using suffixes
pd.merge(reg, login, on = 'name', suffixes = ('_reg', '_log'))

,name,id_reg,id_log
0,Paul,1,3
1,Gilbert,2,4


<a id='string'></a>
## String Methods

In [296]:
# help(str)
names = pd.Series(['andrew','ng','david',12])
names

0    andrew
1        ng
2     david
3        12
dtype: object

In [297]:
names.str.capitalize()

0    Andrew
1        Ng
2     David
3       NaN
dtype: object

In [298]:
names.str.isdigit()

0    False
1    False
2    False
3      NaN
dtype: object

### Splitting, grabbing, expanding

In [299]:
tech_finance = ['GOOG,APPL,AMZN','JPM,BAC,GS']
len(tech_finance)

2

In [300]:
tickers = pd.Series(tech_finance)
tickers

0    GOOG,APPL,AMZN
1        JPM,BAC,GS
dtype: object

In [301]:
tickers.str.split(',')

0    [GOOG, APPL, AMZN]
1        [JPM, BAC, GS]
dtype: object

In [302]:
tickers.str.split(',').str[0]

0    GOOG
1     JPM
dtype: object

In [303]:
tickers.str.split(',',expand=True)

,0,1,2
0,GOOG,APPL,AMZN
1,JPM,BAC,GS


### Cleaning , Editing strings

In [304]:
names = pd.Series(['andrew','da;id','  rose  '])
names

0      andrew
1       da;id
2      rose  
dtype: object

In [305]:
names.str.replace(';','v').str.strip().str.capitalize()

0    Andrew
1     David
2      Rose
dtype: object

In [306]:
# using apply
def clean(name):
  name = name.replace(';','v')
  name = name.strip()
  name = name.capitalize()
  return name

In [307]:
names.apply(clean)

0    Andrew
1     David
2      Rose
dtype: object

In [308]:
# this method is the most efficient
np.vectorize(clean)(names)

array(['Andrew', 'David', 'Rose'], dtype='<U6')

<a id='time'></a>
## Time Methods

In [309]:
pds = pd.Series(['Aug 14, 2018', '2020-02-01', None])
pds

0    Aug 14, 2018
1      2020-02-01
2            None
dtype: object

In [310]:
pd.to_datetime(pds)

0   2018-08-14
1   2020-02-01
2          NaT
dtype: datetime64[ns]

In [311]:
pd.to_datetime(pds)[0]

Timestamp('2018-08-14 00:00:00')

In [312]:
date = '31-12-2020'
pd.to_datetime(date)

Timestamp('2020-12-31 00:00:00')

In [313]:
pd.to_datetime(date, dayfirst=True)

Timestamp('2020-12-31 00:00:00')

### Custom time string 
https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

In [314]:
date_style = '31--Dec--2020'
pd.to_datetime(date_style, format = '%d--%b--%Y')

Timestamp('2020-12-31 00:00:00')

In [315]:
pd.to_datetime('12th of Nov 2020')

Timestamp('2020-11-12 00:00:00')

In [316]:
sales = pd.read_csv('RetailSales_BeerWineLiquor.csv')
sales.head()

,DATE,MRTSSM4453USN
0,1992-01-01,1509
1,1992-02-01,1541
2,1992-03-01,1597
3,1992-04-01,1675
4,1992-05-01,1822


In [317]:
type(sales.iloc[0]['DATE'])

str

In [318]:
sales['DATE'] = pd.to_datetime(sales['DATE'])
sales.iloc[0]['DATE']

Timestamp('1992-01-01 00:00:00')

In [319]:
# parse dates at importing 
sales = pd.read_csv('RetailSales_BeerWineLiquor.csv',parse_dates=[0])
sales.iloc[0]['DATE']

Timestamp('1992-01-01 00:00:00')

In [320]:
# date as index
sales = sales.set_index('DATE')
sales.head()

,MRTSSM4453USN
DATE,
1992-01-01,1509
1992-02-01,1541
1992-03-01,1597
1992-04-01,1675
1992-05-01,1822


### Resample

<table style="display: inline-block">
    <caption style="text-align: center"><strong>TIME SERIES OFFSET ALIASES</strong></caption>
<tr><th>ALIAS</th><th>DESCRIPTION</th></tr>
<tr><td>B</td><td>business day frequency</td></tr>
<tr><td>C</td><td>custom business day frequency (experimental)</td></tr>
<tr><td>D</td><td>calendar day frequency</td></tr>
<tr><td>W</td><td>weekly frequency</td></tr>
<tr><td>M</td><td>month end frequency</td></tr>
<tr><td>SM</td><td>semi-month end frequency (15th and end of month)</td></tr>
<tr><td>BM</td><td>business month end frequency</td></tr>
<tr><td>CBM</td><td>custom business month end frequency</td></tr>
<tr><td>MS</td><td>month start frequency</td></tr>
<tr><td>SMS</td><td>semi-month start frequency (1st and 15th)</td></tr>
<tr><td>BMS</td><td>business month start frequency</td></tr>
<tr><td>CBMS</td><td>custom business month start frequency</td></tr>
<tr><td>Q</td><td>quarter end frequency</td></tr>
<tr><td></td><td><font color=white>intentionally left blank</font></td></tr></table>

<table style="display: inline-block; margin-left: 40px">
<caption style="text-align: center"></caption>
<tr><th>ALIAS</th><th>DESCRIPTION</th></tr>
<tr><td>BQ</td><td>business quarter endfrequency</td></tr>
<tr><td>QS</td><td>quarter start frequency</td></tr>
<tr><td>BQS</td><td>business quarter start frequency</td></tr>
<tr><td>A</td><td>year end frequency</td></tr>
<tr><td>BA</td><td>business year end frequency</td></tr>
<tr><td>AS</td><td>year start frequency</td></tr>
<tr><td>BAS</td><td>business year start frequency</td></tr>
<tr><td>BH</td><td>business hour frequency</td></tr>
<tr><td>H</td><td>hourly frequency</td></tr>
<tr><td>T, min</td><td>minutely frequency</td></tr>
<tr><td>S</td><td>secondly frequency</td></tr>
<tr><td>L, ms</td><td>milliseconds</td></tr>
<tr><td>U, us</td><td>microseconds</td></tr>
<tr><td>N</td><td>nanoseconds</td></tr></table>

In [321]:
sales.resample(rule='A').mean().head()

,MRTSSM4453USN
DATE,
1992-12-31,1807.250000
1993-12-31,1794.833333
1994-12-31,1841.750000
1995-12-31,1833.916667
1996-12-31,1929.750000


### .dt method 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html

In [322]:
sales = sales.reset_index()
# help(sales['DATE'].dt)
sales['DATE'].dt.month

0       1
1       2
2       3
3       4
4       5
       ..
335    12
336     1
337     2
338     3
339     4
Name: DATE, Length: 340, dtype: int64

In [323]:
sales['DATE'].dt.is_leap_year

0       True
1       True
2       True
3       True
4       True
       ...  
335    False
336     True
337     True
338     True
339     True
Name: DATE, Length: 340, dtype: bool